In [ ]:
from random import randint
import random
from pathlib import Path
from PIL import Image, ImageDraw
import pandas as pd
from collections import namedtuple
from fastai.vision.utils import get_image_files
import matplotlib.pyplot as plt
import numpy as np

wbs =  get_image_files(Path("/Users/id/.fastai/data/websites/"))
bss = get_image_files(Path("/Users/id/.fastai/data/kaggle-chess/"))

def path(x): return '/'.join(x.parts[-2:])

df0 = pd.Series(bss, name="board")
df0 = df0.apply(path)

df1 = pd.Series(wbs, name="website")
df1 = df1.repeat(len(df0)//len(df1)+1)[:len(df0)]
df1 = df1.reset_index(drop=True)

np.random.seed(123)
df1=df1.sample(frac=1).reset_index(drop=True)
df1 = df1.apply(path)
df = pd.concat([df0, df1], axis=1, ignore_index=False)

def make_sample():
    w_width = 1024
    w_height = 768
    b_width=400
    b_height=400
    x = randint(50, w_width - b_width - 50)
    y = randint(50, w_height - b_height - 50)
    dx = randint(20, 100)
    dy = randint(20, 100)
    return (x, y, dx, dy)

In [ ]:
import pandas as pd
from pathlib import Path
from PIL import Image, ImageDraw
from random import randint

df = pd.read_csv("annotations.csv")
web_url = Path("/Users/id/.fastai/data/websites/")
board_url = Path("/Users/id/.fastai/data/kaggle-chess/")
gen_url = Path("/Users/id/.fastai/data/chess_screenshots2/")

def make_image(board, website, x, y, dx, dy):
    b = Image.open(board_url/board)
    s = Image.open(web_url/website)
    s.paste(b, (x, y))
    c = s.crop((x-dx, y-dy, x+400+dx, y+400+dy))
    return c

def save_image(img, i): img.save(gen_url / f'{i:06d}.png')
def PILrect(board, website, x, y, dx, dy): return ((dx, dy), (400+dx, 400+dy))

[save_image(make_image(*_[1]), _[0]) for _ in df.iterrows()]


FileNotFoundError: [Errno 2] No such file or directory: 'annotations.csv'